## Bibliotecas

In [1]:
import pandas as pd
from sklearn import tree
from sklearn.model_selection import train_test_split

## constantes

In [ ]:
vogais_num = {
    0 : 'A',
    1 : 'E',
    2 : 'I',
    3 : 'O',
    4 : 'U'
}

# Utilizando dados da sala

## Lendo dados

In [ ]:
db = pd.read_csv('/content/dados_T3_sala.csv', sep=',')
db_previsao = db.copy(deep=True)
## Retirando dados femininos
db = db[db['Genero'] == 1]
db.head()

Locutor  Genero  Vogal   F0    F1    F2    F3
0        1       1      0  110   640  2570  3760
1        1       1      0  100   620  1140  2470
2        1       1      0  100   600  1210  2930
3        1       1      0  110   640  1180  2690
4        1       1      0  510  1130  2460  3590

In [ ]:
media_a_f0 = db[db['Vogal'] == 0]['F0'].mean()
media_a_f1 = db[db['Vogal'] == 0]['F1'].mean()
media_a_f2 = db[db['Vogal'] == 0]['F2'].mean()
media_a_f3 = db[db['Vogal'] == 0]['F3'].mean()

media_e_f0 = db[db['Vogal'] == 1]['F0'].mean()
media_e_f1 = db[db['Vogal'] == 1]['F1'].mean()
media_e_f2 = db[db['Vogal'] == 1]['F2'].mean()
media_e_f3 = db[db['Vogal'] == 1]['F3'].mean()

media_i_f0 = db[db['Vogal'] == 2]['F0'].mean()
media_i_f1 = db[db['Vogal'] == 2]['F1'].mean()
media_i_f2 = db[db['Vogal'] == 2]['F2'].mean()
media_i_f3 = db[db['Vogal'] == 2]['F3'].mean()

media_o_f0 = db[db['Vogal'] == 3]['F0'].mean()
media_o_f1 = db[db['Vogal'] == 3]['F1'].mean()
media_o_f2 = db[db['Vogal'] == 3]['F2'].mean()
media_o_f3 = db[db['Vogal'] == 3]['F3'].mean()

media_u_f0 = db[db['Vogal'] == 4]['F0'].mean()
media_u_f1 = db[db['Vogal'] == 4]['F1'].mean()
media_u_f2 = db[db['Vogal'] == 4]['F2'].mean()
media_u_f3 = db[db['Vogal'] == 4]['F3'].mean()

## Calculando distância

In [ ]:
def calcularDistancia(f1, media_f1, f2, media_f2, f3, media_f3):
  return ((f1 - media_f1)**2 + (f2 - media_f2)**2 + (f3 - media_f3)**2)**(1/2)
## Com f3
def predictVogalWithDistance(f1, f2, f3):
  distancia_a = calcularDistancia(f1, media_a_f1, f2, media_a_f2, f3, media_a_f3)
  distancia_e = calcularDistancia(f1, media_e_f1, f2, media_e_f2, f3, media_e_f3)
  distancia_i = calcularDistancia(f1, media_i_f1, f2, media_i_f2, f3, media_i_f3)
  distancia_o = calcularDistancia(f1, media_o_f1, f2, media_o_f2, f3, media_o_f3)
  distancia_u = calcularDistancia(f1, media_u_f1, f2, media_u_f2, f3, media_u_f3)
  distancias = [distancia_a, distancia_e, distancia_i, distancia_o, distancia_u]
  return distancias.index(min(distancias))
db_previsao['previsão'] = db_previsao.apply(lambda row: predictVogalWithDistance(row['F1'], row['F2'], row['F3']), axis = 1)
def definesTrueOrFalse(real, previsao):
  return real == previsao
db_previsao['True or False'] = db_previsao.apply(lambda row: definesTrueOrFalse(row['Vogal'], row['previsão']), axis=1)
score = round(db_previsao['True or False'].value_counts()[1]/len(db_previsao['True or False'])*100, 2)
print(f'{score}% de acerto')

28.94% de acerto


In [ ]:
db_previsao

Locutor  Genero  Vogal   F0    F1    F2    F3  previsão  True or False
0          1       1      0  110   640  2570  3760         4          False
1          1       1      0  100   620  1140  2470         0           True
2          1       1      0  100   600  1210  2930         0           True
3          1       1      0  110   640  1180  2690         0           True
4          1       1      0  510  1130  2460  3590         4          False
..       ...     ...    ...  ...   ...   ...   ...       ...            ...
420       17       2      4  210  2280  2900  3940         3          False
421       17       2      4  240  3130  4100  5050         3          False
422       17       2      4  470  3050  4230  4930         3          False
423       17       2      4  240  2380  3090  4040         3          False
424       17       2      4  260   780  3100  4130         3          False

[425 rows x 9 columns]

## Com Machine Learning

In [ ]:
training_data, testing_data = train_test_split(db, test_size=0.2, random_state=25)
training_data_without_resp = training_data.drop(labels='Vogal', axis=1)
clf = tree.DecisionTreeClassifier()
clf = clf.fit(training_data_without_resp, training_data['Vogal'])

In [ ]:
testing_data_without_resp = testing_data.drop(labels='Vogal', axis=1)
testing_data['previsão ml'] = clf.predict(testing_data_without_resp)
def definesTrueOrFalse(real, previsao):
  return real == previsao
testing_data['True or False ml'] = testing_data.apply(lambda row: definesTrueOrFalse(row['Vogal'], row['previsão ml']), axis=1)
score = round(testing_data['True or False ml'].value_counts()[1]/len(testing_data['True or False ml'])*100, 2)
print(f'{score}% de acerto')

58.75% de acerto


# Utilizando dados da sala + base do professor

## Lendo dados

In [ ]:
db = pd.read_csv('/content/Dados_T3.csv', sep=';')
db_previsao = db.copy(deep=True)
## Retirando dados femininos
db = db[db['Genero'] == 1]
db.head()

Genero  Locutor  Tentativa  Fonema   f0   f1    f2
0       1        1          1       0  203  608  1216
1       1        1          2       0  231  574  1146
2       1        1          3       0  157  627  1251
3       1        2          1       0  123  743  1235
4       1        2          2       0  139  823  1226

In [ ]:
media_a_f0 = db[db['Fonema'] == 0]['f0'].mean()
media_a_f1 = db[db['Fonema'] == 0]['f1'].mean()
media_a_f2 = db[db['Fonema'] == 0]['f2'].mean()

media_e_f0 = db[db['Fonema'] == 1]['f0'].mean()
media_e_f1 = db[db['Fonema'] == 1]['f1'].mean()
media_e_f2 = db[db['Fonema'] == 1]['f2'].mean()

media_i_f0 = db[db['Fonema'] == 2]['f0'].mean()
media_i_f1 = db[db['Fonema'] == 2]['f1'].mean()
media_i_f2 = db[db['Fonema'] == 2]['f2'].mean()

media_o_f0 = db[db['Fonema'] == 3]['f0'].mean()
media_o_f1 = db[db['Fonema'] == 3]['f1'].mean()
media_o_f2 = db[db['Fonema'] == 3]['f2'].mean()

media_u_f0 = db[db['Fonema'] == 4]['f0'].mean()
media_u_f1 = db[db['Fonema'] == 4]['f1'].mean()
media_u_f2 = db[db['Fonema'] == 4]['f2'].mean()

## Comparando F1 e F2

In [ ]:
def calcularDistancia(f1, media_f1, f2, media_f2):
  return ((float(f1) - float(media_f1))**2 + (float(f2) - float(media_f2))**2)**(1/2)
def predictVogalWithDistanceF1andF2(f1, f2):
  distancia_a = calcularDistancia(f1, media_a_f1, f2, media_a_f2)
  distancia_e = calcularDistancia(f1, media_e_f1, f2, media_e_f2)
  distancia_i = calcularDistancia(f1, media_i_f1, f2, media_i_f2)
  distancia_o = calcularDistancia(f1, media_o_f1, f2, media_o_f2)
  distancia_u = calcularDistancia(f1, media_u_f1, f2, media_u_f2)
  distancias = [distancia_a, distancia_e, distancia_i, distancia_o, distancia_u]
  return distancias.index(min(distancias))
db_previsao['previsão f1 e f2'] = db_previsao.apply(lambda row: predictVogalWithDistanceF1andF2(row['f1'],row['f2']), axis = 1)
def definesTrueOrFalse(real, previsao):
  return float(str(real).replace(",", ".")) == float(str(previsao).replace(",", "."))
db_previsao['True or False f1 e f2'] = db_previsao.apply(lambda row: definesTrueOrFalse(row['Fonema'], row['previsão f1 e f2']), axis=1)
db_previsao['True or False f1 e f2']
score = round(db_previsao['True or False f1 e f2'].value_counts()[1]/len(db_previsao['True or False f1 e f2'])*100, 2)
print(f'{score}% de acerto')

34.5% de acerto


In [ ]:
db_previsao

Genero  Locutor  Tentativa  Fonema   f0    f1    f2  previsão f1 e f2  \
0         1        1          1       0  203   608  1216                 0   
1         1        1          2       0  231   574  1146                 0   
2         1        1          3       0  157   627  1251                 0   
3         1        2          1       0  123   743  1235                 0   
4         1        2          2       0  139   823  1226                 0   
..      ...      ...        ...     ...  ...   ...   ...               ...   
995       2       54          1       4  210  2280  2900                 2   
996       2       54          2       4  240  3130  4100                 2   
997       2       54          3       4  470  3050  4230                 2   
998       2       54          4       4  240  2380  3090                 2   
999       2       54          5       4  260   780  3100                 2   

     True or False f1 e f2  
0                     True  
1                     True  
2                     True  
3                     True  
4                     True  
..                     ...  
995                  False  
996                  False  
997                  False  
998                  False  
999                  False  

[1000 rows x 9 columns]

## Com Machine Learning

In [ ]:
training_data, testing_data = train_test_split(db, test_size=0.2, random_state=25)
training_data_without_resp = training_data.drop(labels='Fonema', axis=1)
clf = tree.DecisionTreeClassifier()
clf = clf.fit(training_data_without_resp, training_data['Fonema'])

In [ ]:
testing_data_without_resp = testing_data.drop(labels='Fonema', axis=1)
testing_data['previsão ml'] = clf.predict(testing_data_without_resp)
def definesTrueOrFalse(real, previsao):
  return real == previsao
testing_data['True or False ml'] = testing_data.apply(lambda row: definesTrueOrFalse(row['Fonema'], row['previsão ml']), axis=1)
score = round(testing_data['True or False ml'].value_counts()[1]/len(testing_data['True or False ml'])*100, 2)
print(f'{score}% de acerto')

75.13% de acerto


## Passando F1 e F2 de um áudio individual
Com código feito no Matlab/Octave, retiramos f0, f1 e f2

In [ ]:
# Dados retirados do código Matlab (get_formantes.m) com um áudio
f1 = 790
f2 = 1290
vogais_num[predictVogalWithDistanceF1andF2(f1, f2)]

'A'